In [1]:
#필요한 패키지로드
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from glob import glob
import librosa
import warnings

warnings.filterwarnings("ignore")

In [2]:
sample_submission = pd.read_csv("./acc/sample_submission.csv")

africa_train_paths = glob("./acc/train/africa/*.wav")
australia_train_paths = glob("./acc/train/australia/*.wav")
canada_train_paths = glob("./acc/train/canada/*.wav")
england_train_paths = glob("./acc/train/england/*.wav")
hongkong_train_paths = glob("./acc/train/hongkong/*.wav")
us_train_paths = glob("./acc/train/us/*.wav")

path_list = [africa_train_paths, australia_train_paths, canada_train_paths,
             england_train_paths, hongkong_train_paths, us_train_paths]

In [3]:
# glob로 test data의 path를 불러올때 순서대로 로드되지 않을 경우를 주의해야 합니다.
# test_ 데이터 프레임을 만들어서 나중에 sample_submission과 id를 기준으로 merge시킬 준비를 합니다.

def get_id(data):
    return data.split("/")[3].split(".")[0]

test_ = pd.DataFrame(index = range(0, 6100), columns = ["path", "id"])
test_["path"] = glob("./acc/test/*.wav")
test_["id"] = test_["path"].apply(lambda x : get_id(x))

test_.head()

,path,id
0,./acc/test/1636.wav,1636
1,./acc/test/2045.wav,2045
2,./acc/test/3766.wav,3766
3,./acc/test/576.wav,576
4,./acc/test/5634.wav,5634


In [7]:
def load_data(paths):

    result = []
    for path in tqdm(paths):
        # sr = 16000이 의미하는 것은 1초당 16000개의 데이터를 샘플링 한다는 것입니다.
        data, sr = librosa.load(path, sr = 16000)
        result.append(data)
    result = np.array(result) 
    # 메모리가 부족할 때는 데이터 타입을 변경해 주세요 ex) np.array(data, dtype = np.float32)

    return result

In [54]:
# train 데이터를 로드하기 위해서는 많은 시간이 소모 됩니다.
# 따라서 추출된 정보를 npy파일로 저장하여 필요 할 때마다 불러올 수 있게 준비합니다.

os.mkdir("./npy_data")

africa_train_data = load_data(africa_train_paths)
np.save("./npy_data/africa_npy", africa_train_data)

australia_train_data = load_data(australia_train_paths)
np.save("./npy_data/australia_npy", australia_train_data)

canada_train_data = load_data(canada_train_paths)
np.save("./npy_data/canada_npy", canada_train_data)

england_train_data = load_data(england_train_paths)
np.save("./npy_data/england_npy", england_train_data)

hongkong_train_data = load_data(hongkong_train_paths)
np.save("./npy_data/hongkong_npy", hongkong_train_data)

us_train_data = load_data(us_train_paths)
np.save("./npy_data/us_npy", us_train_data)

test_data = load_data(test_["path"])
np.save("./npy_data/test_npy", test_data)

100%|██████████| 6100/6100 [18:17<00:00,  5.56it/s]


In [12]:
# npy파일로 저장된 데이터를 불러옵니다.
africa_train_data = np.load("./npy_data/africa_npy.npy", allow_pickle = True)
australia_train_data = np.load("./npy_data/australia_npy.npy", allow_pickle = True)
canada_train_data = np.load("./npy_data/canada_npy.npy", allow_pickle = True)
england_train_data = np.load("./npy_data/england_npy.npy", allow_pickle = True)
hongkong_train_data = np.load("./npy_data/hongkong_npy.npy", allow_pickle = True)
us_train_data = np.load("./npy_data/us_npy.npy", allow_pickle = True)

test_data = np.load("./npy_data/test_npy.npy", allow_pickle = True)

train_data_list = [africa_train_data, australia_train_data, canada_train_data, england_train_data, hongkong_train_data, us_train_data]

In [57]:
train_data_list[0][1].shape

(114816,)

In [16]:
africa_train_data[1].shape

(114816,)

In [6]:
# 이번 대회에서 음성은 각각 다른 길이를 갖고 있습니다.
# baseline 코드에서는 음성 중 길이가 가장 작은 길이의 데이터를 기준으로 데이터를 잘라서 사용합니다.

def get_mini(data):

    mini = 9999999
    for i in data:
        if len(i) < mini:
            mini = len(i)

    return mini

#음성들의 길이를 맞춰줍니다.

def set_length(data, d_mini):

    result = []
    for i in data:
        result.append(i[:d_mini])
    result = np.array(result)

    return result

#feature를 생성합니다.

def get_feature(data, sr = 16000, n_fft = 256, win_length = 200, hop_length = 160, n_mels = 64):
    mel = []
    for i in data:
        # win_length 는 음성을 작은 조각으로 자를때 작은 조각의 크기입니다.
        # hop_length 는 음성을 작은 조각으로 자를때 자르는 간격을 의미합니다.
        # n_mels 는 적용할 mel filter의 개수입니다.
        mel_ = librosa.feature.melspectrogram(i, sr = sr, n_fft = n_fft, win_length = win_length, hop_length = hop_length, n_mels = n_mels)
        mel.append(mel_)
    mel = np.array(mel)
    mel = librosa.power_to_db(mel, ref = np.max)

    mel_mean = mel.mean()
    mel_std = mel.std()
    mel = (mel - mel_mean) / mel_std

    return mel

In [52]:
#feature-mel 
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import numpy as np
from librosa.feature import melspectrogram

__author__ = 'Konstantinos Drossos -- Tampere University'
__docformat__ = 'reStructuredText'
__all__ = ['feature_extraction']


def feature_extraction(audio_data: np.ndarray,
                       sr: int,
                       nb_fft: int,
                       hop_size: int,
                       nb_mels: int,
                       f_min: float,
                       f_max: float,
                       htk: bool,
                       power: float,
                       norm: bool,
                       window_function: str,
                       center: bool)\
        -> np.ndarray:
    """Feature extraction function.

    :param audio_data: Audio signal.
    :type audio_data: numpy.ndarray
    :param sr: Sampling frequency.
    :type sr: int
    :param nb_fft: Amount of FFT points.
    :type nb_fft: int
    :param hop_size: Hop size in samples.
    :type hop_size: int
    :param nb_mels: Amount of MEL bands.
    :type nb_mels: int
    :param f_min: Minimum frequency in Hertz for MEL band calculation.
    :type f_min: float
    :param f_max: Maximum frequency in Hertz for MEL band calculation.
    :type f_max: float|None
    :param htk: Use the HTK Toolbox formula instead of Auditory toolkit.
    :type htk: bool
    :param power: Power of the magnitude.
    :type power: float
    :param norm: Area normalization of MEL filters.
    :type norm: bool
    :param window_function: Window function.
    :type window_function: str
    :param center: Center the frame for FFT.
    :type center: bool
    :return: Log mel-bands energies of shape=(t, nb_mels)
    :rtype: numpy.ndarray
    """
    y = audio_data
    mel_bands = melspectrogram(
        y=y, sr=sr, n_fft=nb_fft, hop_length=hop_size, win_length=nb_fft,
        window=window_function, center=center, power=power, n_mels=nb_mels,
        fmin=f_min, fmax=f_max, htk=htk, norm=norm).T
    logmel_spectrogram = librosa.core.power_to_db(
            mel_bands, ref=1.0, amin=1e-10, 
            top_db=None)
    logmel_spectrogram = logmel_spectrogram.astype(np.float32)        
    return logmel_spectrogram

# EOF


,

''

In [53]:
process={
'sr':16000,
'nb_fft':1024,
'hop_size': 512,
'nb_mels':64,
'window_function':'hann',
'center':True,
'f_min':.0,
'f_max':None,
'htk':False,
'power':1,
'norm':1}

In [83]:



def load_numpy_object(file_name: Path)\
        -> Union[np.ndarray, np.recarray]:
    """Loads and returns a numpy object.

    :param file_name: File name of the numpy object.
    :type file_name: pathlib.Path
    :return: Numpy object.
    :rtype: numpy.ndarray|numpy.rec.array
    """
    return np.load(str(file_name), allow_pickle=True)

NameError: name 'Path' is not defined

In [54]:
train=[]
for i in train_x:
    x_train=feature_extraction(i,**process)
    train.append(x_train)
    

In [60]:
train[2].shape

(264, 64)

In [82]:
train

[array([[-100.      , -100.      , -100.      , ..., -100.      ,
         -100.      , -100.      ],
        [-100.      , -100.      , -100.      , ..., -100.      ,
         -100.      , -100.      ],
        [ -29.36471 ,  -33.597977,  -36.974808, ...,  -43.051514,
          -43.9159  ,  -49.426956],
        ...,
        [ -36.23842 ,  -37.10531 ,  -36.652943, ...,  -44.229275,
          -44.410355,  -48.084698],
        [ -33.57678 ,  -40.78979 ,  -39.395306, ...,  -43.638702,
          -43.373314,  -47.841232],
        [ -37.832623,  -38.254498,  -38.626846, ...,  -44.1536  ,
          -45.46821 ,  -49.351162]], dtype=float32),
 array([[-100.      , -100.      , -100.      , ..., -100.      ,
         -100.      , -100.      ],
        [-100.      , -100.      , -100.      , ..., -100.      ,
          -98.41084 ,  -96.43733 ],
        [ -35.203392,  -33.910873,  -33.867634, ...,  -39.94906 ,
          -38.595554,  -42.557602],
        ...,
        [ -12.93409 ,  -17.202696,  -20

In [43]:
len(train_x)

25520

In [42]:
x_train.shape

NameError: name 'x_train' is not defined

In [37]:
train_x

array([array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
       array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
       array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), ...,
       array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        5.0592377e-07, -5.9019806e-07,  6.2548929e-07], dtype=float32),
       array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
       -8.6025248e-06, -1.7511389e-06,  1.7679191e-06], dtype=float32),
       array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)], dtype=object)

In [20]:
train_x = np.concatenate(train_data_list, axis= 0)
test_x = np.array(test_data)

In [69]:
train_x[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [7]:
train_x = np.concatenate(train_data_list, axis= 0)
test_x = np.array(test_data)

# 음성의 길이 중 가장 작은 길이를 구합니다.

train_mini = get_mini(train_x)
test_mini = get_mini(test_x)

mini = np.min([train_mini, test_mini])

# data의 길이를 가장 작은 길이에 맞춰 잘라줍니다.

train_x = set_length(train_x, mini)
test_x = set_length(test_x, mini)

# librosa를 이용해 feature를 추출합니다.

train_x = get_feature(data = train_x)
test_x = get_feature(data = test_x)

train_x = train_x.reshape(-1, train_x.shape[1], train_x.shape[2], 1)
test_x = test_x.reshape(-1, test_x.shape[1], test_x.shape[2], 1)


In [8]:
test_x.shape

(6100, 64, 501, 1)

In [9]:
train_x.shape

(25520, 64, 501, 1)

In [62]:
#라벨링
train_y = np.concatenate((np.zeros(len(africa_train_data), dtype = np.int),
                        np.ones(len(australia_train_data), dtype = np.int),
                         np.ones(len(canada_train_data), dtype = np.int) * 2,
                         np.ones(len(england_train_data), dtype = np.int) * 3,
                         np.ones(len(hongkong_train_data), dtype = np.int) * 4,
                         np.ones(len(us_train_data), dtype = np.int) * 5), axis = 0)

In [63]:

train_y = pd.get_dummies(train_y).to_numpy(dtype = 'long')

In [90]:
from typing import Tuple, List, AnyStr, Union
from pathlib import Path

from numpy import ndarray, recarray
from torch.utils.data import Dataset
from numpy import load as np_load
import pickle
import torch
import numpy as np
import os
from torch import cat as pt_cat, zeros as pt_zeros, \
    ones as pt_ones, from_numpy, Tensor
from torch.utils.data.dataloader import DataLoader

In [66]:
class ToTensor(object):
    """numpy array를 tensor(torch)로 변환합니다."""
    def __call__(self, sample):
        x, y = sample['x'], sample['y']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        x = x.transpose((2, 0, 1))
        return {'x': torch.FloatTensor(x),
                'y': torch.FloatTensor(y)}
to_tensor = transforms.Compose([
                      ToTensor() 
])


class CustomDataset(torch.utils.data.Dataset): 
    def __init__(self,train_x,train_y,transforms=to_tensor):
        self.x_data = train_x
        self.y_data = train_y
        self.transforms = transforms# Transform

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx): 
        
        x = self.x_data[idx]
        y = self.y_data[idx]
        sample = {'x': x, 'y': y}
        if self.transforms:
            sample = self.transforms(sample)
        y = y.astype(np.float32)
        
        
        return (x,y)
    
def collate_fn(batch,input_pad_at = 'start'):

    in_t_steps = max([i[0].shape[0] for i in batch])

    in_dim = batch[0][0].shape[-1]

    input_tensor, output_tensor = [], []

    for in_b, out_b in batch:
        if in_t_steps >= in_b.shape[0]:
            padding = pt_zeros(in_t_steps - in_b.shape[0], in_dim).float()
            data = [from_numpy(in_b).float()]
            if input_pad_at.lower() == 'start':
                data.insert(0, padding)
            else:
                data.append(padding)
            tmp_in: Tensor = pt_cat(data)
        else:
            tmp_in: Tensor = from_numpy(in_b[:in_t_steps, :]).float()
        input_tensor.append(tmp_in.unsqueeze_(0))
        tmp_out: Tensor = torch.Tensor(out_b)
        output_tensor.append(tmp_out.unsqueeze_(0))


    input_tensor = pt_cat(input_tensor)
    output_tensor = pt_cat(output_tensor)

    return input_tensor, output_tensor

def acc_loader(batch_size,shuffle=True,drop_last=True):
    dataset = CustomDataset(train_x,train_y)
    return DataLoader(
        dataset=dataset, batch_size=batch_size,
        shuffle=shuffle, drop_last=drop_last, collate_fn=collate_fn)

In [67]:
train_data=acc_loader(batch_size=256)
bar  = tqdm(train_data,total=len(train_data))

  0%|          | 0/99 [00:00<?, ?it/s]

In [69]:
for feature, tag in enumerate(bar):
        print("feature:",feature)
        print("tag:",tag)

ValueError: axes don't match array

In [65]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms
import timm

In [73]:
#gpu 할당
device='cuda'

In [74]:
class MultiLabelResnet(nn.Module):
    def __init__(self):
        super(MultiLabelResnet, self).__init__()
        self.conv2d = nn.Conv2d(64, 3, 1, stride=1)
        self.resnet = timm.create_model('resnet101', pretrained=False) 
        self.FC = nn.Linear(1000, 6)

    def forward(self, x):
        # resnet의 입력은 [3, N, N]으로
        # 3개의 채널을 갖기 때문에
        # resnet 입력 전에 conv2d를 한 층 추가
        x = F.relu(self.conv2d(x))

        # resnet18을 추가
        x = F.relu(self.resnet(x))

        # 마지막 출력에 nn.Linear를 추가
        # multilabel을 예측해야 하기 때문에
        # softmax가 아닌 sigmoid를 적용
        #x = self.FC(x)
        x = torch.sigmoid(self.FC(x))
        return x
# 모델 선언

model = MultiLabelResnet()
model.to(device)

MultiLabelResnet(
  (conv2d): Conv2d(64, 3, kernel_size=(1, 1), stride=(1, 1))
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=

In [72]:
device

NameError: name 'device' is not defined

In [32]:
dataset = CustomDataset(train_x,train_y)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

In [38]:
dataset[0][0].shape#64 mel , 길이 501 


(64, 501, 1)

In [40]:
dataloader

TypeError: 'DataLoader' object does not support indexing

In [75]:
with tqdm(dataloader,total=len(dataloader)) as bar:
        for idx,sample in enumerate(bar):
            print(sample[0])

NameError: name 'dataloader' is not defined

In [76]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
criterion  = torch.nn.BCELoss()

In [31]:
dataloader.__len__()

100

In [77]:
from tqdm import tqdm

# 모델의 dropoupt, batchnormalization를 train 모드로 설정
model.train()

for epoch in range(1,10+1):
    # 1개 epoch 훈련
    train_acc_list = []
    with tqdm(train_data,#train_data_loader를 iterative하게 반환
            total=train_data.__len__(), # train_data_loader의 크기
            unit="batch") as train_bar: # 한번 반환하는 smaple의 단위는 "batch"
        for idx,sample in enumerate(train_bar):
            if idx == 224 :
                break
            train_bar.set_description(f"Train Epoch {epoch}")
            # 갱신할 변수들에 대한 모든 변화도를 0으로 초기화
            # 참고)https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html
            optimizer.zero_grad()

            images, labels = sample
            # tensor를 gpu에 올리기 
            images = images.to(device)
            labels = labels.to(device)


            
            # .forward()에서 중간 노드의 gradient를 계산
            with torch.set_grad_enabled(True):
                # 모델 예측
                probs = model(images)
                probs = F.softmax(probs)
               # probs = (probs == probs.max()) * 1.0
                #loss = criterion(probs, y_train)
                
                
                loss = criterion(probs, labels)
                #loss = criterion(probs, torch.max(y_train, 1)[1])

                loss.backward()
                optimizer.step()
                
                probs  = probs.cpu().detach().numpy()
                labels = labels.cpu().detach().numpy()
                # train accuracy 계산
                cnt = 0
                for i in range(0,256):
                    if probs[i].argmax() == labels[i].argmax():
                    
                        cnt +=1
                        
        
                #preds = probs > 0.5
                #batch_acc = (labels == preds).mean()
                batch_acc = cnt/256 #(배치사이즈로 나누기)
                train_acc_list.append(batch_acc)
                train_acc = np.mean(train_acc_list)

            # 현재 progress bar에 현재 미니배치의 loss 결과 출력
            train_bar.set_postfix(train_loss= loss.item(),
                                    train_acc = train_acc)


  0%|          | 0/99 [00:00<?, ?batch/s]


ValueError: axes don't match array

In [ ]:
train_

In [56]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(train_x, train_y)
train_dataloader = DataLoader(train_dataset, batch_size=256)

for x, y in train_dataloader:
    print (x.shape)

TypeError: 'int' object is not callable

In [58]:
train_x.shape

(25520, 64, 501, 1)

In [25]:
with tqdm(dataloader,#train_data_loader를 iterative하게 반환
            total=dataloader.__len__(), # train_data_loader의 크기
            unit="batch") as train_bar: # 한번 반환하는 smaple의 단위는 "batch"
        for idx,sample in enumerate(train_bar):
            if idx == 224 :
                break
            train_bar.set_description(f"Train Epoch {epoch}")
            # 갱신할 변수들에 대한 모든 변화도를 0으로 초기화
            # 참고)https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html
            optimizer.zero_grad()

            images, labels = sample
            # tensor를 gpu에 올리기 
            images = images.to(device)
            labels = labels.to(device)
        

Train Epoch 1: 100%|██████████| 100/100 [00:01<00:00, 55.39batch/s]


In [ ]:
def train(epoch):
    # bar  = tqdm(training_data,total=len(training_data))
    loss_list = []
    model.train()
    with tqdm(training_data,total=len(training_data)) as bar:
        for i, (feature, tag) in enumerate(bar):
            feature = feature.to(device)
            tag = tag.to(device)
            optimizer.zero_grad()
            out_tag = model(feature)
            loss = tag_loss(out_tag,tag)
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
            bar.set_description("epoch:{} idx:{} loss:{:.6f}".format(epoch, i, np.mean(loss_list)))
    return np.mean(loss_list)

def test(epoch):
    eva_loss = []
    model.eval()
    with torch.no_grad():
        for i, (feature, tag) in enumerate(test_data):
            feature = feature.to(device)
            tag = tag.to(device)
            out_tag = model(feature)
            loss = tag_loss(out_tag, tag)
            eva_loss.append(loss.item())
    mean_loss = np.mean(eva_loss)
    print("epoch:{:d}--testloss:{:.6f}".format(epoch,mean_loss.item()))

    # return  mean_loss


if __name__ == '__main__':
    train_b = True
    epoch_last = 0
    if train_b:
        # model.load_state_dict(torch.load("./models/280/TagModel_{}.pt".format(str(40))))
        for epoch in range(epoch_last+1,epoch_last+5):
            train(epoch)
            scheduler.step(epoch)
            test(epoch)
            if epoch%5==0:
                torch.save(model.state_dict(),'./models/tag_models/TagModel_{}.pt'.format(epoch))
    else:
        for epoch in range(0,225,5):
            model.load_state_dict(torch.load("./models/tag_models/TagModel_{}.pt".format(str(epoch))))
            test(epoch)



In [27]:
images.shape

torch.Size([176, 64, 501, 1])

In [30]:
len(train_bar)

100

In [54]:
probs.shape

torch.Size([212, 6])

In [55]:
probs[179]

tensor([0.4080, 0.4133, 0.3688, 0.7125, 0.3799, 0.7079], device='cuda:0',
       grad_fn=<SelectBackward>)

In [62]:
probs = model(images)

In [65]:
probs.argmax

tensor([[0.4402, 0.3849, 0.4081, 0.6494, 0.3744, 0.7220],
        [0.4264, 0.3729, 0.4102, 0.6647, 0.3673, 0.7419],
        [0.4028, 0.3277, 0.3589, 0.7337, 0.3404, 0.8159],
        ...,
        [0.4398, 0.3776, 0.3898, 0.6565, 0.3651, 0.7151],
        [0.4323, 0.3492, 0.3824, 0.6936, 0.3503, 0.7744],
        [0.4264, 0.3916, 0.4190, 0.6495, 0.3813, 0.7322]], device='cuda:0',
       grad_fn=<SigmoidBackward>)

In [33]:
range(10)

range(0, 10)

In [48]:
test_x.shape

(6100, 64, 501, 1)

In [35]:
model.eval()
test_y = pd.DataFrame(index=range(0,len(test_x)), columns=['0', '1', '2', '3', '4', '5'])
test_y = test_y.fillna(0).to_numpy()
dataset = CustomDataset(test_x,test_y)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)


In [49]:
len(dataloader)

24

In [43]:
test_y.shape

(6100, 6)

In [36]:
pred_ = []
for idx, sample in enumerate(dataloader):
    with torch.no_grad():
        # 추론
        model.eval()
        images,_ = sample
        images = images.to(device)
        probs  = model(images)
        probs = F.softmax(probs)
        probs = probs.cpu().detach().numpy()
        pred_.append(probs)
        


In [47]:
probs.shape

(212, 6)

In [37]:
pred_

[array([[0.15157057, 0.15375239, 0.14690813, 0.19858463, 0.15054806,
         0.19863619],
        [0.15023679, 0.15304187, 0.14553908, 0.20272414, 0.14728282,
         0.20117536],
        [0.14998494, 0.15468429, 0.14548811, 0.20463553, 0.14651199,
         0.19869515],
        ...,
        [0.1495984 , 0.15174147, 0.14601216, 0.20251873, 0.14862378,
         0.20150548],
        [0.15158302, 0.1554691 , 0.14678638, 0.198491  , 0.15035564,
         0.19731481],
        [0.15330476, 0.15870261, 0.14831421, 0.19460163, 0.15203936,
         0.19303748]], dtype=float32),
 array([[0.1481286 , 0.15474829, 0.14566131, 0.20342004, 0.14660773,
         0.2014341 ],
        [0.14698723, 0.15581152, 0.14398839, 0.20303325, 0.1473341 ,
         0.20284556],
        [0.1515528 , 0.15458037, 0.14757988, 0.19696695, 0.15020427,
         0.19911568],
        ...,
        [0.14821756, 0.15302   , 0.14563133, 0.2041979 , 0.14497176,
         0.20396143],
        [0.1497958 , 0.15209864, 0.14711751, 0.

In [38]:
def cov_type(data):
    return np.int(data)

# 처음에 살펴본 것처럼 glob로 test data의 path는 sample_submission의 id와 같이 1,2,3,4,5.....으로 정렬 되어있지 않습니다.
# 만들어둔 test_ 데이터프레임을 이용하여 sample_submission과 predict값의 id를 맞춰줍니다.
sample_submission = pd.read_csv("./acc/sample_submission.csv")


In [39]:
sample_submission

,id,africa,australia,canada,england,hongkong,us
0,1,0,0,0,0,0,0
1,2,0,0,0,0,0,0
2,3,0,0,0,0,0,0
3,4,0,0,0,0,0,0
4,5,0,0,0,0,0,0
...,...,...,...,...,...,...,...
6095,6096,0,0,0,0,0,0
6096,6097,0,0,0,0,0,0
6097,6098,0,0,0,0,0,0
6098,6099,0,0,0,0,0,0


In [41]:
result = pd.concat([test_, pd.DataFrame(np.mean(pred, axis = 0))], axis = 1).iloc[:, 1:]


NameError: name 'pred' is not defined

In [39]:
np.mean(pred_)

ValueError: operands could not be broadcast together with shapes (256,6) (212,6) 